## CRUD Operations on Redis using Python

For this example, we are using [Redis](https://hub.docker.com/_/redis) image in Docker compose environment.

### Spin up Docker compose environment

In [42]:
!docker compose up --remove-orphans -d 

# --remove-orphans will remove any previously ran dangling containers
# -d will run container in a detached mode

Network redis-python_default  Creating
Network redis-python_default  Created
Container redis-python-db-1  Creating
Container redis-python-db-1  Created
Container redis-python-db-1  Starting
Container redis-python-db-1  Started


### Install Redis library for Python

In [43]:
pip install redis

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: C:\Users\prakh\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


### Create a client connection

In [44]:
from redis import Redis

HOST = 'localhost'
PORT = 6379
DATABASE = 0
PASSWORD = 'simplepass'

client = Redis(host=HOST, port=PORT, db=DATABASE, password=PASSWORD)

if client is not None and client.ping():
    print('Client Connection Successful !!')
else:
    print('Client Connection Failed (X)')

Client Connection Successful !!


### Set and Get an item using Client

In [45]:
KEY1 = "some-key-1"
VALUE1 = "some-value-1"

client_set_result = client.set(KEY1, VALUE1)
print(client_set_result)
client_get_result = client.get(KEY1)
print(client_get_result)

True
b'some-value-1'


### Create a Client using Connection Pool

In [46]:
from redis import ConnectionPool

connection_pool = ConnectionPool(host=HOST, port=PORT, db=DATABASE, password=PASSWORD)
pool_client = Redis(connection_pool=connection_pool)

if pool_client is not None and pool_client.ping():
    print('Pool Client Connection Successful !!')
else:
    print('Pool Client Connection Failed (X)')

Pool Client Connection Successful !!


### Set and Get an item using Pool Client

In [47]:
KEY2 = "some-key-2"
VALUE2 = "some-value-2"

pool_client_set_result = pool_client.set(KEY2, VALUE2)
print(pool_client_set_result)
pool_client_get_result = pool_client.get(KEY2)
print(pool_client_get_result)

True
b'some-value-2'


### Create an AsyncIO client

In [48]:
import redis.asyncio
from datetime import datetime

start_time = datetime.now()
asyncio_client = redis.asyncio.Redis(host=HOST, port=PORT, db=DATABASE, password=PASSWORD)
await asyncio_client.ping()
end_time = datetime.now()
print(f"AsyncIO connection Established in : {end_time - start_time} seconds")

AsyncIO connection Established in : 0:00:02.010646 seconds


### Set and Get an item using AsyncIO client

In [49]:
KEY3 = "some-key-3"
VALUE3 = "some-value-3"

asyncio_client_set_result = asyncio_client.set(KEY3, VALUE3)
print(f"{await asyncio_client_set_result}")
asyncio_client_get_result = asyncio_client.get(KEY3)
print(f"{await asyncio_client_get_result}")

True
b'some-value-3'


### Transactions with AsyncIO client

In [50]:
KEY4 = "some-key-4"
VALUE4 = "some-value-4"
KEY5 = "some-key-5"
VALUE5 = "some-value-5"

transaction_asyncio_client = await redis.asyncio.from_url(f"redis://:{PASSWORD}@{HOST}:{PORT}/{DATABASE}")

async with transaction_asyncio_client.pipeline(transaction=True) as pipeline:
    ok1, ok2 = await (
            pipeline \
            .set(KEY4, VALUE4) \
            .set(KEY5, VALUE5) \
            .execute()
        )

assert ok1
assert ok2

### Create a Pub/Sub Client

In [51]:
import asyncio
import async_timeout

STOPWORD = "STOP"

async def reader(channel: redis.asyncio.client.PubSub):
    while True:
        try:
            async with async_timeout.timeout(1):
                message = await channel.get_message(ignore_subscribe_messages=True)
                if message is not None:
                    print(f"(Reader) Message Received: {message}")
                    if message["data"].decode() == STOPWORD:
                        print("(Reader) STOP")
                        break
                await asyncio.sleep(0.01)
        except asyncio.TimeoutError:
            pass

pubsub_client = redis.asyncio.from_url(f"redis://:{PASSWORD}@{HOST}:{PORT}/{DATABASE}")
pubsub = pubsub_client.pubsub()
await pubsub.subscribe("channel:1", "channel:2")

future = asyncio.create_task(reader(pubsub))

await pubsub_client.publish("channel:1", "Hello")
await pubsub_client.publish("channel:2", "World")
await pubsub_client.publish("channel:1", STOPWORD)

await future

(Reader) Message Received: {'type': 'message', 'pattern': None, 'channel': b'channel:1', 'data': b'Hello'}
(Reader) Message Received: {'type': 'message', 'pattern': None, 'channel': b'channel:2', 'data': b'World'}
(Reader) Message Received: {'type': 'message', 'pattern': None, 'channel': b'channel:1', 'data': b'STOP'}
(Reader) STOP


### Clean up connections and resources

In [52]:
client.close()
pool_client.close()
asyncio_client.close()
transaction_asyncio_client.close()
pubsub_client.close()

!docker compose down

C:\Users\prakh\AppData\Local\Temp\ipykernel_19160\1209903389.py:3: RuntimeWarning: coroutine 'Redis.close' was never awaited
  asyncio_client.close()
C:\Users\prakh\AppData\Local\Temp\ipykernel_19160\1209903389.py:4: RuntimeWarning: coroutine 'Redis.close' was never awaited
  transaction_asyncio_client.close()
C:\Users\prakh\AppData\Local\Temp\ipykernel_19160\1209903389.py:5: RuntimeWarning: coroutine 'Redis.close' was never awaited
  pubsub_client.close()
Container redis-python-db-1  Stopping
Container redis-python-db-1  Stopping
Container redis-python-db-1  Stopped
Container redis-python-db-1  Removing
Container redis-python-db-1  Removed
Network redis-python_default  Removing
Network redis-python_default  Removed
